In [1]:
import micromagneticmodel as mm
import oommfc as mc

import discretisedfield as df  # df is here chosen to be an alias for discretisedfield

p1 = (0, 0, 0)
p2 = (100e-9, 50e-9, 20e-9)

region = df.Region(p1=p1, p2=p2)

In [2]:
import pyvista as pv

pv.set_jupyter_backend("trame")

In [3]:
pv.__version__

'0.41.1'

In [4]:
lx, ly, lz = 100e-9, 50e-9, 20e-9

subregions = {
    "bottom_subregion": df.Region(p1=(20e-9, 0, 0), p2=(40e-9, 50e-9, 10e-9)),
    "top_subregion": df.Region(p1=(80e-9, 40e-9, lz / 2), p2=(lx, ly, lz)),
}

cell = (5e-9, 5e-9, 5e-9)

region = df.Region(p1=(0, 0, 0), p2=(lx, ly, lz))
mesh = df.Mesh(region=region, cell=cell, subregions=subregions)

In [5]:
plotter = pv.Plotter()
mesh.pyvista(plotter=plotter, wireframe=True)
plotter.show()

Widget(value="<iframe src='http://localhost:40295/index.html?ui=P_0x7f1c53f13f40_0&reconnect=auto' style='widt…

In [50]:
a, b, c = 1e-9, 1e-9, 1e-9
cell = (0.5e-9, 0.5e-9, 0.5e-9)

mesh = df.Mesh(p1=(-a, -b, -c), p2=(a, b, c), n=(5, 5, 5))  # cell=cell)


def norm_fun(pos):
    x, y, z = pos
    if (x / a) ** 2 + (y / b) ** 2 + (z / c) ** 2 <= 1:
        return 1e6
    else:
        return 0


def value_fun(pos):
    x, y, z = pos
    c = 1e9
    return (-c * y, c * x, c * z)


field = df.Field(mesh, nvdim=3, value=value_fun, norm=norm_fun, valid="norm")
field

Field(Mesh(Region(pmin=[-1e-09, -1e-09, -1e-09], pmax=[1e-09, 1e-09, 1e-09], dims=['x', 'y', 'z'], units=['m', 'm', 'm']), n=[5, 5, 5]), nvdim=3, vdims: (x, y, z))

In [44]:
import numpy as np


def test_field():
    mesh = df.Mesh(p1=(-5e-9, -5e-9, -5e-9), p2=(5e-9, 5e-9, 5e-9), n=(5, 5, 5))
    c_array = mesh.coordinate_field().array

    # The norm is defined via numpy for performance reasons;
    # In the simple loop form it would be:
    # x, y, _ = point
    # if x**2 + y**2 <= 5e-9**2:
    #     return 1e5
    # else:
    #     return 0
    def norm(points):
        return np.where(
            (points[..., 0] ** 2 + points[..., 1] ** 2) <= 5e-9**2, 1e5, 0
        )[..., np.newaxis]

    # Values are defined in numpy for performance reasons
    # We define vector fields with vx=0, vy=0, vz=+/-1 for x<0 / x>0
    def value(points):
        res = np.zeros((*mesh.n, 3))
        res[..., 2] = np.where(points[..., 0] <= 0, 1, -1)
        return res

    return df.Field(
        mesh,
        nvdim=3,
        value=value(c_array),
        norm=norm(c_array),
        vdims=["x", "y", "z"],
        valid=True,  #  "norm",
    )

In [46]:
field = test_field()
field

Field(Mesh(Region(pmin=[-5e-09, -5e-09, -5e-09], pmax=[5e-09, 5e-09, 5e-09], dims=['x', 'y', 'z'], units=['m', 'm', 'm']), n=[5, 5, 5]), nvdim=3, vdims: (x, y, z))

In [49]:
field.x

Field(Mesh(Region(pmin=[-5e-09, -5e-09, -5e-09], pmax=[5e-09, 5e-09, 5e-09], dims=['x', 'y', 'z'], units=['m', 'm', 'm']), n=[5, 5, 5]), nvdim=1)

In [51]:
field.x

Field(Mesh(Region(pmin=[-1e-09, -1e-09, -1e-09], pmax=[1e-09, 1e-09, 1e-09], dims=['x', 'y', 'z'], units=['m', 'm', 'm']), n=[5, 5, 5]), nvdim=1)

In [47]:
field.pyvista.valid()

Widget(value="<iframe src='http://localhost:40295/index.html?ui=P_0x7f1b501fc490_21&reconnect=auto' style='wid…

In [42]:
field.x.pyvista.contour(isosurfaces=5)

Widget(value="<iframe src='http://localhost:40295/index.html?ui=P_0x7f1b480f5610_20&reconnect=auto' style='wid…

In [12]:
field.angle((1, 0, 0))

/home/holtsamu/repos/ubermag-devtools/repos/discretisedfield/discretisedfield/field.py:1242: RuntimeWarning: invalid value encountered in divide
  res_array = function(self.array, other)


Field(Mesh(Region(pmin=[-1e-09, -1e-09, -1e-09], pmax=[1e-09, 1e-09, 1e-09], dims=['x', 'y', 'z'], units=['m', 'm', 'm']), n=[4, 4, 4]), nvdim=1, unit=rad)

In [38]:
field.pyvista.contour(contour_kwargs={"scalars": "x"})

ValueError: Empty meshes cannot be plotted. Input mesh has zero points.

In [14]:
field.y.pyvista.contour()

Widget(value="<iframe src='http://localhost:40295/index.html?ui=P_0x7f1c327eba30_5&reconnect=auto' style='widt…

In [9]:
field_pv = pv.wrap(field.to_vtk())
field_pv

RectilinearGrid (0x7f9c6be3e640)
  N Cells:      1920
  N Points:     2457
  X Bounds:     -5.000e+00, 5.000e+00
  Y Bounds:     -3.000e+00, 3.000e+00
  Z Bounds:     -2.000e+00, 2.000e+00
  Dimensions:   21, 13, 9
  N Arrays:     6

In [10]:
pv.wrap(field.x.to_vtk())

RectilinearGrid (0x7f9c6be3e580)
  N Cells:      1920
  N Points:     2457
  X Bounds:     -5.000e+00, 5.000e+00
  Y Bounds:     -3.000e+00, 3.000e+00
  Z Bounds:     -2.000e+00, 2.000e+00
  Dimensions:   21, 13, 9
  N Arrays:     3

In [11]:
field_pv["color_field"] = pv.wrap(field.x.to_vtk())["field"].copy()
field_pv

RectilinearGrid (0x7f9c6be3e640)
  N Cells:      1920
  N Points:     2457
  X Bounds:     -5.000e+00, 5.000e+00
  Y Bounds:     -3.000e+00, 3.000e+00
  Z Bounds:     -2.000e+00, 2.000e+00
  Dimensions:   21, 13, 9
  N Arrays:     7

In [12]:
field.pyvista.vector()

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c6be32df0_2&reconnect=auto' style='widt…

In [13]:
field.pyvista.vector(cmap="viridis", color_field=field.y)

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9cd80d49a0_3&reconnect=auto' style='widt…

In [14]:
field.pyvista.vector(cmap="viridis", scalars="x")

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c6bbd0040_4&reconnect=auto' style='widt…

In [15]:
field.x.pyvista.scalar(opacity=[1, 0, 1])

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9cd80d4190_5&reconnect=auto' style='widt…

In [16]:
field.pyvista.contour(isosurfaces=5)

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c6e48baf0_6&reconnect=auto' style='widt…

In [17]:
field.pyvista.contour(isosurfaces=10, scalars="y")

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c6bbd4c70_7&reconnect=auto' style='widt…

In [21]:
field.pyvista.streamlines()

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c28060250_11&reconnect=auto' style='wid…

In [22]:
field.pyvista.valid(opacity=1, culling="backface")

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c28066790_12&reconnect=auto' style='wid…

In [20]:
plotter = pv.Plotter()
field.pyvista.vector(plotter=plotter, cmap="viridis")
field.pyvista.contour(plotter=plotter, isosurfaces=10, scalars="x")
plotter.show()

Widget(value="<iframe src='http://localhost:39181/index.html?ui=P_0x7f9c280662e0_10&reconnect=auto' style='wid…